## AChecker.ai

#### AChecker.ai is an accessibility checker that uses Selenium, web scraping, and the OpenAI API.


##### It:
1. Loads a given web page in headless Chrome.
2. Extracts the page’s HTML.
3. Sends that content to GPT with prompts based on WCAG 2 AA Standards.
4. Returns structured JSON with a summary, category, and a list of issues, each including:
    - Issue description
    - Location (CSS-selector-style path)
    - Severity (low/medium/high)
    - Recommendation to fix

Selenium is used instead of plain requests so the page is rendered like a real browser (including JavaScript) before analysis.

##### JSON Response
```
{
    "website": "website url",
    "quick_summary": "brief and concise summary of the website content",
    "category": "Website category",
    "issues": [
      {
        "issue": "issue description (use WCAG 2 AA standards)",
        "location": "the exact html element where the issue is exists e.g div(class=x, id=y or p.span : file line number (e.g line 23))",
        "severity": "severity of the issue (low, medium, high)",
        "recommendation": "recommendation for fixing the issue"
      }
    ]
  }
```

##### Additional packages to install
- selenium
- webdriver-manager

In [1]:
# imports

import os

from IPython.display import display, Markdown, JSON
from dotenv import load_dotenv
from openai import OpenAI
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
load_dotenv(override=True)

if (os.getenv("OPENAI_API_KEY") is None):
    raise ValueError("OPENAI_API_KEY is not set")
    



In [3]:

"""
    Code Inspiration Credit: community-contributions/bishara-selenium-fix
"""
def load_website(url):
    print(f"Fetching content from {url} using Selenium...")
    driver = None
    
    try:
        print("1. Setting up Chrome options...")
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--disable-gpu")
        chrome_options.add_argument("window-size=1200x600")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")

        print("2. Installing/finding WebDriver...")
        driver_path = ChromeDriverManager().install()
        service = Service(driver_path)
        
        print("3. Initializing WebDriver...")
        driver = webdriver.Chrome(service=service, options=chrome_options)
        
        print("4. Setting page load timeout...")
        driver.set_page_load_timeout(20)

        print(f"5. Getting URL: {url}...")
        driver.get(url)
        
        print("6. Setting wait...")
        body_element = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )
        
        print("7. Finding body content...")
        body_content = body_element.get_attribute("outerHTML")
        
        print("8. Content fetched successfully.")
        print("--------------------------------------------------------------------")

        return body_content
    
    except Exception as e:
        print(f"9. Error during Selenium operation: {e}")
        if driver:
            body_content = driver.find_element(By.TAG_NAME, "body").get_attribute("outerHTML")
            if body_content:
                print("10. Page timed out, but returning partial content.")
                return body_content
        return None
    finally:
        if driver:
            print("11. Shutting down WebDriver...")
            driver.close()
            driver.quit()
            print("12. WebDriver shut down.")

In [6]:
 
 # Step 1: Create your prompts

system_prompt = """
  Parse this website for accessibility issues 
  and provide a list of the issues found. 
  Read and use WCAG 2 AA standards(https://www.w3.org/TR/WCAG22/).
  Ensure you are using the correct WCAG 2 AA standards for the issue and be assertive.
  Respond with the following JSON format:
  {
    "website": "website url",
    "quick_summary": "brief and concise summary of the website content",
    "category": "Website category",
    "issues": [
      {
        "issue": "issue description (use WCAG 2 AA standards)",
        "location": "CSS selector of the element where the issue is exists e.g div.class, span#id or ul:nth-child(2). Selectors only and should be as specific as possible",
        "severity": "severity of the issue (low, medium, high)",
        "recommendation": "recommendation for fixing the issue"
      }
    ]
  }
"""

user_prompt = """
  Parse this website for accessibility issues and provide a list of the issues found.
"""

# Step 2: Make the messages list

def get_messages(website):
  website_content = load_website(website)
  if not website_content:
    return "Sorry, I couldn't fetch the website. It's probably broken or my robot eyes can't see it."

  return [
      {"role": "system", "content": system_prompt},
      {"role": "user", "content": f"{user_prompt} website content: {website_content}"}
  ]

# fill this in

# Step 3: Call OpenAI
def analyze_website(website):
  openai = OpenAI()
  response = openai.chat.completions.create(
    model="gpt-5-nano", 
    messages=get_messages(website),
    response_format={"type": "json_object"}
  )
  print("13: Response received from OpenAI")
  return response.choices[0].message.content

# Step 4: print the result
website = "https://edwarddonner.com/curriculum/"
output = analyze_website(website)



Fetching content from https://edwarddonner.com/curriculum/ using Selenium...
1. Setting up Chrome options...
2. Installing/finding WebDriver...
3. Initializing WebDriver...
4. Setting page load timeout...
5. Getting URL: https://edwarddonner.com/curriculum/...
6. Setting wait...
7. Finding body content...
8. Content fetched successfully.
--------------------------------------------------------------------
11. Shutting down WebDriver...
12. WebDriver shut down.
13: Response received from OpenAI


In [7]:
# Step 5: Display the result
print(output)




{
  "website": "https://edwarddonner.com/curriculum/",
  "quick_summary": "The AI Curriculum page presents a six-course curriculum with a featured image, a detailed course table, multiple contact forms, and media embeds (image gallery and YouTube playlist).",
  "category": "Education / AI Curriculum / Personal Portfolio",
  "issues": [
    {
      "issue": "Document language not declared (WCAG 3.1.1 Language of the Web) – HTML element missing a lang attribute",
      "location": "html",
      "severity": "high",
      "recommendation": "Add a language attribute to the root element, e.g., <html lang='en'>, to ensure screen readers know the document language."
    },
    {
      "issue": "YouTube embed iframe is missing a descriptive title (WCAG 2.4.3 Focus Order and WCAG 1.1.1 Non-text Content) ",
      "location": "iframe.youtube-player",
      "severity": "high",
      "recommendation": "Add a descriptive title attribute (for example, title='AI Curriculum YouTube Playlist') or an acce